## 1.优化算法进阶

### 1.1 Momentum

均以函数f(x)=0.1* x1^2  +  2* x2^2为例

In [1]:
def f_2d(x1, x2):
    return 0.1 * x1 ** 2 + 2 * x2 ** 2
def momentum_2d(x1, x2, v1, v2):
    v1 = beta * v1 + eta * 0.2 * x1
    v2 = beta * v2 + eta * 4 * x2
    return x1 - v1, x2 - v2, v1, v2

### 1.2 AdaGrad 

后期学习率过小可能难以找到有用的解

In [2]:
def adagrad_2d(x1, x2, s1, s2):
    g1, g2, eps = 0.2 * x1, 4 * x2, 1e-6  # 前两项为自变量梯度
    s1 += g1 ** 2
    s2 += g2 ** 2
    x1 -= eta / math.sqrt(s1 + eps) * g1
    x2 -= eta / math.sqrt(s2 + eps) * g2
    return x1, x2, s1, s2

#### 1.3 RMSProp 

AdaGrad的改进

In [3]:
def rmsprop_2d(x1, x2, s1, s2):
    g1, g2, eps = 0.2 * x1, 4 * x2, 1e-6
    s1 = beta * s1 + (1 - beta) * g1 ** 2
    s2 = beta * s2 + (1 - beta) * g2 ** 2
    x1 -= alpha / math.sqrt(s1 + eps) * g1
    x2 -= alpha / math.sqrt(s2 + eps) * g2
    return x1, x2, s1, s2

### 1.4 AdaDelta 

同样为AdaGrad的改进

In [4]:
def adadelta_2d(x1,x2,s1,s2,xt1,xt2):
    g1, g2, eps = 0.2 * x1, 4 * x2, 1e-6
    s1 = beta * s1 + (1 - beta) * g1 ** 2
    s2 = beta * s2 + (1 - beta) * g2 ** 2
    g1_other=math.sqrt((xt1+eps)/(s1+eps))*g1
    g2_other=math.sqrt((xt2+eps)/(s1+eps))*g2
    x1-=g1_other
    x2-=g2_other
    xt1=alpha*xt1+(1-alpha)*g1_other**2
    xt2=alpha*xt2+(1-alpha)*g2_other**2
    return x1,x2,s1,s2,xt1,xt2

### 1.5Adam 

In [5]:
def adam(x1, x2, m1, m2, v1, v2):
    g1, g2, eps = 0.2 * x1, 4 * x2, 1e-6  # 前两项为自变量梯度
    m1=beta1*m1+(1-beta1)*g1
    m2=beta1*m2+(1-beta1)*g2
    v1=beta2*v1+(1-beta2)*g1**2
    v2=beta2*v2+(1-beta2)*g2**2
    m1_other=m1/(1-beta1_t)
    m2_other=m2/(1-beta1_t)
    v1_other=v1/(1-beta2_t)
    v2_other=v2/(1-beta2_t)
    g1_other=eta*m1_other/(math.sqrt(v1_other)+eps)
    g2_other=eta*m2_other/(math.sqrt(v2_other)+eps)
    x1-=g1_other
    x1-=g2_other
    return x1,x2,m1,m2,v1,v2

## 2.数据增强

### 2.1图像增广

图像增广（image augmentation）技术通过对训练图像做一系列随机改变，来产生相似但又不同的训练样本，从而扩大训练数据集的规模。图像增广的另一种解释是，随机改变训练样本可以降低模型对某些属性的依赖，从而提高模型的泛化能力。下面介绍常用的方法

### 翻转和裁剪

In [8]:
import torchvision
#随机水平翻转
aug=torchvision.transforms.RandomHorizontalFlip()
#随机垂直翻转
aug=torchvision.transforms.RandomVerticalFlip()
#每次随机裁剪出一块面积为原面积 10%∼100% 的区域，且该区域的宽和高之比随机取自 0.5∼2 ，
#然后再将该区域的宽和高分别缩放到200像素。
shape_aug = torchvision.transforms.RandomResizedCrop(200, scale=(0.1, 1), ratio=(0.5, 2))

### 变化颜色
亮度、对比度、饱和度、色调

In [9]:
#图像亮度随机变华为原亮度的50%(1-0.5至1+0.5)
aug=torchvision.transforms.ColorJitter(brightness=0.5, contrast=0, saturation=0, hue=0)
#色调
aug=torchvision.transforms.ColorJitter(brightness=0, contrast=0, saturation=0, hue=0.5)
#对比度
aug=torchvision.transforms.ColorJitter(brightness=0, contrast=0.5, saturation=0, hue=0)
#同时改变亮度（brightness）、对比度（contrast）、饱和度（saturation）和色调（hue）
color_aug = torchvision.transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5)

### 叠加多个图像增广方法

In [10]:
augs = torchvision.transforms.Compose([torchvision.transforms.RandomHorizontalFlip(), color_aug, shape_aug])

## 3.模型微调

(1)在源数据集（如ImageNet数据集）上预训练一个神经网络模型，即源模型。

(2)创建一个新的神经网络模型，即目标模型。它复制了源模型上除了输出层外的所有模型设计及其参数。我们假设这些模型参数包含了源数据集上学习到的知识，且这些知识同样适用于目标数据集。我们还假设源模型的输出层跟源数据集的标签紧密相关，因此在目标模型中不予采用。
(3)为目标模型添加一个输出大小为目标数据集类别个数的输出层，并随机初始化该层的模型参数。

(4)在目标数据集（如椅子数据集）上训练目标模型。我们将从头训练输出层，而其余层的参数都是基于源模型的参数微调得到的

In [1]:
%matplotlib inline
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision import models
import os
import sys

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
###加载热狗数据集
data_dir = '/home/kesci/input/hotdog4014'
train_imgs = ImageFolder(os.path.join(data_dir, 'hotdog/train'))
test_imgs = ImageFolder(os.path.join(data_dir, 'hotdog/test'))
#从图像中裁剪出随机大小和随机高宽比的一块随机区域，然后将该区域缩放为高和宽均为224像素的输入。
#测试时，我们将图像的高和宽均缩放为256像素，然后从中裁剪出高和宽均为224像素的中心区域作为输入。
#此外，我们对RGB（红、绿、蓝）三个颜色通道的数值做标准化：每个数值减去该通道所有数值的平均值，再除以该通道所有数值的标准差作为输出。
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_augs = transforms.Compose([
        transforms.RandomResizedCrop(size=224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ])

test_augs = transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        normalize
    ])

In [ ]:
#定义和初始化预训练模型
pretrained_net = models.resnet18(pretrained=False)
pretrained_net.load_state_dict(torch.load('/home/kesci/input/resnet185352/resnet18-5c106cde.pth'))
print(pretrained_net.fc)
#源模型输出类别中由1000个，本文中只需要使用2个，修改输出的fc层
pretrained_net.fc = nn.Linear(512, 2)
print(pretrained_net.fc)

pretrained_net的fc层就被随机初始化了，但是其他层依然保存着预训练得到的参数。

由于是在很大的ImageNet数据集上预训练的，所以参数已经足够好，因此一般只需使用较小的学习率来微调这些参数，而fc中的随机初始化参数一般需要更大的学习率从头训练。

PyTorch可以方便的对模型的不同部分设置不同的学习参数，我们在下面代码中将fc的学习率设为已经预训练过的部分的10倍。

In [ ]:
output_params = list(map(id, pretrained_net.fc.parameters()))
feature_params = filter(lambda p: id(p) not in output_params, pretrained_net.parameters())

lr = 0.01
optimizer = optim.SGD([{'params': feature_params},
                       {'params': pretrained_net.fc.parameters(), 'lr': lr * 10}],
                       lr=lr, weight_decay=0.001)

In [ ]:
#训练模型
def train_fine_tuning(net, optimizer, batch_size=128, num_epochs=5):
    train_iter = DataLoader(ImageFolder(os.path.join(data_dir, 'hotdog/train'), transform=train_augs),
                            batch_size, shuffle=True)
    test_iter = DataLoader(ImageFolder(os.path.join(data_dir, 'hotdog/test'), transform=test_augs),
                           batch_size)
    loss = torch.nn.CrossEntropyLoss()
    train(train_iter, test_iter, net, loss, optimizer, device, num_epochs)
    train_fine_tuning(pretrained_net, optimizer)